# PHS564 — Lecture 08 (Student)
## IP weighting for confounding (propensity scores) + diagnostics (MIMIC-IV Demo)

### Learning goals
- Explain IP weighting as **creating a pseudo-population** where treatment is independent of measured confounders.
- Fit a propensity model \(e(L)=Pr(A=1|L)\) and compute **unstabilized** and **stabilized** weights.
- Diagnose failure modes: **lack of overlap/positivity**, extreme weights, imbalance after weighting.
- Produce a defensible causal estimate + a minimal diagnostics panel.

### Required reading
- Hernán & Robins, *What If*, Chapter 12 (IPW basics + intuition): https://miguelhernan.org/whatifbook


### Setup

This notebook is designed to run **locally** or in **Google Colab**.

**Colab workflow (recommended):**
1) Clone the course repo (ask the instructor for the GitHub URL).
2) Install requirements.
3) Run the notebook top-to-bottom.

> If you opened this notebook directly from GitHub in Colab (without cloning),
> relative paths will not work. Clone first.


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
RNG = np.random.default_rng(564)

# Locate repo root (works when running from lectures/Lxx.../student or /instructor)
THIS_DIR = Path.cwd()
REPO_ROOT = THIS_DIR
for _ in range(4):
    if (REPO_ROOT / "requirements.txt").exists() or (REPO_ROOT / "README.md").exists():
        break
    REPO_ROOT = REPO_ROOT.parent

DATA_DIR = REPO_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"

print("Working directory:", THIS_DIR)
print("Repo root:", REPO_ROOT)
print("Processed data dir exists:", PROC_DIR.exists())


## Data
This lecture uses a **pre-built cohort extract** from MIMIC-IV Demo.

Expected file:
- `data/processed/cohort_L08_ps_ipw.parquet` (preferred) or `.csv`

If missing, run:
```bash
python data/download_data.py
```
(or ask the instructor for the course data bundle).


In [ ]:
# Statsmodels for regression (logit/ols); installed via requirements.txt
import statsmodels.api as sm
import statsmodels.formula.api as smf
# Load cohort (parquet preferred, fallback to csv)
parquet_path = PROC_DIR / "cohort_L08_ps_ipw.parquet"
csv_path = PROC_DIR / "cohort_L08_ps_ipw.csv"

if parquet_path.exists():
    df = pd.read_parquet(parquet_path)
elif csv_path.exists():
    df = pd.read_csv(csv_path)
else:
    raise FileNotFoundError(
        "Cohort file not found. Expected one of:\n"
        f" - {parquet_path}\n - {csv_path}\n"
        "Run `python data/download_data.py` or ask the instructor for data."
    )

df.head()

### Choose variables
We will define:
- Treatment/exposure `A`
- Outcome `Y`
- Baseline covariates/confounders `L_list`

**Instructor should pre-specify** these for the cohort (see slides). Students should only edit the covariate list and model formulas.


In [ ]:
# TODO: set these to match the cohort
A = "A"   # treatment column name
Y = "Y"   # outcome column name (binary or continuous)
L_list = ["age", "sex"]  # TODO: extend with confounders available in df

# Quick sanity checks
missing = [c for c in [A,Y,*L_list] if c not in df.columns]
missing

## Part A — Propensity score model


In [ ]:
# Drop rows with missing values in analysis variables
analysis_vars = [A, Y] + L_list
d = df[analysis_vars].dropna().copy()

# TODO: specify the PS model. Keep it simple and explicit.
# Example: "A ~ age + sex + diabetes + ... "
ps_formula = A + " ~ " + " + ".join(L_list)
ps_model = smf.logit(ps_formula, data=d).fit(disp=False)

d["ps"] = ps_model.predict(d)

# Stabilized weights
pA = d[A].mean()
d["sw"] = np.where(d[A]==1, pA/d["ps"], (1-pA)/(1-d["ps"]))

d[["ps","sw"]].describe(percentiles=[0.01,0.05,0.95,0.99])

### TODO A1 — Weight diagnostics + truncation
Plot the weights. Choose a truncation rule (e.g., 1st/99th percentile) and create `sw_trunc`.


In [ ]:
plt.figure()
plt.hist(d["sw"], bins=60)
plt.xlabel("stabilized weight")
plt.ylabel("count")
plt.title("Weights (before truncation)")
plt.show()

# TODO: truncation
lo, hi = d["sw"].quantile([0.01, 0.99]).to_list()
d["sw_trunc"] = d["sw"].clip(lower=lo, upper=hi)

plt.figure()
plt.hist(d["sw_trunc"], bins=60)
plt.xlabel("truncated weight")
plt.ylabel("count")
plt.title("Weights (after truncation)")
plt.show()

(lo, hi)

## Part B — IPW estimate of effect
We compute weighted mean outcomes by A. For binary Y, this estimates a risk difference.


In [ ]:
def wmean(x,w): 
    x = np.asarray(x); w = np.asarray(w)
    return np.sum(w*x)/np.sum(w)

mu1 = wmean(d.loc[d[A]==1, Y], d.loc[d[A]==1, "sw_trunc"])
mu0 = wmean(d.loc[d[A]==0, Y], d.loc[d[A]==0, "sw_trunc"])
ate = mu1 - mu0
{"mu1": float(mu1), "mu0": float(mu0), "ATE (mu1-mu0)": float(ate)}

### TODO B1 — Sensitivity: compare to crude estimate


In [ ]:
mu1_crude = d.loc[d[A]==1, Y].mean()
mu0_crude = d.loc[d[A]==0, Y].mean()
ate_crude = mu1_crude - mu0_crude
{"crude": float(ate_crude), "ipw": float(ate)}

## Reflection
1) What does a large difference between crude and IPW imply?
2) Which assumption is most vulnerable in EHR data: exchangeability, positivity, or consistency?
